# 🧹 Étape 2 : Nettoyage et Préparation des Données

## 📋 Objectifs de cette étape
1. **Charger** les données explorées de l'étape 1
2. **Traiter** les valeurs aberrantes intelligemment
3. **Encoder** les variables catégorielles
4. **Ingénierie** des fonctionnalités (feature engineering)
5. **Diviser** les données (train/validation/test)
6. **Normaliser** les données pour la modélisation

---

## 🛠️ Configuration et Imports

In [1]:
# Configuration générale
import warnings
warnings.filterwarnings('ignore')

# Manipulation des données
import pandas as pd
import numpy as np
from pathlib import Path

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Statistiques
from scipy import stats
from scipy.stats import zscore

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# Paramètres d'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Configuration terminée !")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📊 NumPy version: {np.__version__}")
print(f"🤖 Scikit-learn importé avec succès")

✅ Configuration terminée !
📦 Pandas version: 2.3.2
📊 NumPy version: 2.3.3
🤖 Scikit-learn importé avec succès


## 📊 Chargement des Données Explorées

Nous allons charger les données nettoyées de l'étape 1 d'exploration.

In [2]:
# 1. Chargement des données depuis l'étape 1
processed_data_path = Path('../data/processed')

# Vérifier que les fichiers existent
original_file = processed_data_path / 'housing_original.csv'
processed_file = processed_data_path / 'housing_processed.csv'

if original_file.exists() and processed_file.exists():
    # Charger les deux versions
    df_original = pd.read_csv(original_file)
    df_processed = pd.read_csv(processed_file)
    
    print("✅ Données chargées avec succès !")
    print(f"📊 Dataset original: {df_original.shape}")
    print(f"📊 Dataset traité: {df_processed.shape}")
    
    # Utiliser le dataset traité comme base
    df = df_processed.copy()
    
else:
    print("❌ Fichiers de données non trouvés !")
    print("💡 Assurez-vous d'avoir exécuté le notebook 01_data_exploration.ipynb")
    
    # Plan B : charger directement depuis le fichier source
    print("🔄 Chargement depuis le fichier source...")
    source_path = Path('../data/Housing Prices Dataset/Housing.csv')
    
    if source_path.exists():
        df_original = pd.read_csv(source_path)
        
        # Appliquer les transformations de base
        df = df_original.copy()
        
        # Variables binaires yes/no -> 1/0
        binary_vars = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 
                       'airconditioning', 'prefarea']
        
        for var in binary_vars:
            df[var] = df[var].map({'yes': 1, 'no': 0})
        
        # Variable furnishingstatus -> numérique
        furnishing_map = {'furnished': 2, 'semi-furnished': 1, 'unfurnished': 0}
        df['furnishingstatus'] = df['furnishingstatus'].map(furnishing_map)
        
        print("✅ Données chargées et transformées !")
    else:
        print("❌ Fichier source non trouvé non plus !")

# 2. Aperçu des données chargées
if 'df' in locals():
    print(f"\n🔍 APERÇU DES DONNÉES CHARGÉES:")
    print(f"   Shape: {df.shape}")
    print(f"   Variables numériques: {df.select_dtypes(include=[np.number]).shape[1]}")
    print(f"   Variables catégorielles: {df.select_dtypes(include=['object']).shape[1]}")
    print(f"   Valeurs manquantes: {df.isnull().sum().sum()}")
    
    print(f"\n📋 VARIABLES DISPONIBLES:")
    for i, col in enumerate(df.columns, 1):
        col_type = "Numérique" if df[col].dtype in ['int64', 'float64'] else "Catégorielle"
        print(f"   {i:2d}. {col:18s} ({col_type})")

✅ Données chargées avec succès !
📊 Dataset original: (545, 14)
📊 Dataset traité: (545, 13)

🔍 APERÇU DES DONNÉES CHARGÉES:
   Shape: (545, 13)
   Variables numériques: 13
   Variables catégorielles: 0
   Valeurs manquantes: 0

📋 VARIABLES DISPONIBLES:
    1. price              (Numérique)
    2. area               (Numérique)
    3. bedrooms           (Numérique)
    4. bathrooms          (Numérique)
    5. stories            (Numérique)
    6. mainroad           (Numérique)
    7. guestroom          (Numérique)
    8. basement           (Numérique)
    9. hotwaterheating    (Numérique)
   10. airconditioning    (Numérique)
   11. parking            (Numérique)
   12. prefarea           (Numérique)
   13. furnishingstatus   (Numérique)


## 🚨 Traitement Intelligent des Valeurs Aberrantes

Nous allons traiter les outliers de manière intelligente en analysant s'ils sont des erreurs ou des valeurs légitimes.

In [3]:
# Fonctions utilitaires pour les outliers
def detect_outliers_iqr(data, column):
    """Détecte les outliers avec la méthode IQR"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

def detect_outliers_zscore(data, column, threshold=3):
    """Détecte les outliers avec la méthode Z-score"""
    z_scores = np.abs(zscore(data[column]))
    outliers = data[z_scores > threshold]
    return outliers, z_scores

def detect_outliers_isolation_forest(data, contamination=0.1):
    """Détecte les outliers avec Isolation Forest"""
    # Sélectionner seulement les variables numériques
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    X = data[numeric_cols]
    
    iso_forest = IsolationForest(contamination=contamination, random_state=42)
    outliers_pred = iso_forest.fit_predict(X)
    
    # -1 pour outliers, 1 pour normaux
    outliers_mask = outliers_pred == -1
    outliers = data[outliers_mask]
    
    return outliers, outliers_mask

print("✅ Fonctions de détection d'outliers définies !")

✅ Fonctions de détection d'outliers définies !


In [4]:
# 1. Analyse détaillée des outliers
print("🔍 ANALYSE DÉTAILLÉE DES VALEURS ABERRANTES")
print("=" * 50)

# Variables à analyser
variables_to_analyze = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
outliers_summary = {}

for var in variables_to_analyze:
    if var in df.columns:
        # Méthode IQR
        outliers_iqr, lower_iqr, upper_iqr = detect_outliers_iqr(df, var)
        
        # Méthode Z-score
        outliers_zscore, z_scores = detect_outliers_zscore(df, var)
        
        outliers_summary[var] = {
            'iqr_count': len(outliers_iqr),
            'zscore_count': len(outliers_zscore),
            'iqr_bounds': (lower_iqr, upper_iqr),
            'outliers_indices': set(outliers_iqr.index)
        }
        
        print(f"\n📊 {var.upper()}:")
        print(f"   IQR Outliers: {len(outliers_iqr)} ({len(outliers_iqr)/len(df)*100:.1f}%)")
        print(f"   Z-score Outliers: {len(outliers_zscore)} ({len(outliers_zscore)/len(df)*100:.1f}%)")
        print(f"   IQR Seuils: [{lower_iqr:.1f}, {upper_iqr:.1f}]")
        
        if len(outliers_iqr) > 0:
            print(f"   Valeurs extrêmes: {outliers_iqr[var].min():.1f} - {outliers_iqr[var].max():.1f}")

# 2. Analyse globale avec Isolation Forest
print(f"\n🤖 DÉTECTION GLOBALE - ISOLATION FOREST:")
outliers_iso, outliers_mask = detect_outliers_isolation_forest(df, contamination=0.05)
print(f"   Outliers globaux détectés: {len(outliers_iso)} ({len(outliers_iso)/len(df)*100:.1f}%)")

# 3. Analyse des propriétés avec outliers multiples
print(f"\n🏠 PROPRIÉTÉS AVEC OUTLIERS MULTIPLES:")
all_outlier_indices = set()
for var, info in outliers_summary.items():
    all_outlier_indices.update(info['outliers_indices'])

if all_outlier_indices:
    outlier_properties = df.loc[list(all_outlier_indices)]
    print(f"   Total propriétés avec outliers: {len(all_outlier_indices)}")
    print(f"\n   Exemples de propriétés extrêmes:")
    display_cols = ['price', 'area', 'bedrooms', 'bathrooms']
    print(outlier_properties[display_cols].head())
    
    # Prix moyens avec vs sans outliers
    prix_avec_outliers = outlier_properties['price'].mean()
    prix_sans_outliers = df.drop(all_outlier_indices)['price'].mean()
    print(f"\n   Prix moyen avec outliers: {prix_avec_outliers:,.0f}")
    print(f"   Prix moyen sans outliers: {prix_sans_outliers:,.0f}")
    print(f"   Différence: {abs(prix_avec_outliers - prix_sans_outliers):,.0f}")

🔍 ANALYSE DÉTAILLÉE DES VALEURS ABERRANTES

📊 PRICE:
   IQR Outliers: 15 (2.8%)
   Z-score Outliers: 6 (1.1%)
   IQR Seuils: [-35000.0, 9205000.0]
   Valeurs extrêmes: 9240000.0 - 13300000.0

📊 AREA:
   IQR Outliers: 12 (2.2%)
   Z-score Outliers: 7 (1.3%)
   IQR Seuils: [-540.0, 10500.0]
   Valeurs extrêmes: 10700.0 - 16200.0

📊 BEDROOMS:
   IQR Outliers: 12 (2.2%)
   Z-score Outliers: 2 (0.4%)
   IQR Seuils: [0.5, 4.5]
   Valeurs extrêmes: 5.0 - 6.0

📊 BATHROOMS:
   IQR Outliers: 1 (0.2%)
   Z-score Outliers: 11 (2.0%)
   IQR Seuils: [-0.5, 3.5]
   Valeurs extrêmes: 4.0 - 4.0

📊 STORIES:
   IQR Outliers: 41 (7.5%)
   Z-score Outliers: 0 (0.0%)
   IQR Seuils: [-0.5, 3.5]
   Valeurs extrêmes: 4.0 - 4.0

📊 PARKING:
   IQR Outliers: 12 (2.2%)
   Z-score Outliers: 0 (0.0%)
   IQR Seuils: [-1.5, 2.5]
   Valeurs extrêmes: 3.0 - 3.0

🤖 DÉTECTION GLOBALE - ISOLATION FOREST:

📊 PRICE:
   IQR Outliers: 15 (2.8%)
   Z-score Outliers: 6 (1.1%)
   IQR Seuils: [-35000.0, 9205000.0]
   Valeurs extrê

In [5]:
# 4. Décision de traitement des outliers
print("⚖️ DÉCISION DE TRAITEMENT DES OUTLIERS")
print("=" * 45)

# Créer une copie pour les tests
df_clean = df.copy()
outliers_removed = 0

# Règles de traitement intelligentes
treatment_rules = {
    'price': {'action': 'cap', 'reason': 'Prix extrêmes possibles dans l\'immobilier'},
    'area': {'action': 'cap', 'reason': 'Grandes propriétés possibles'},
    'bedrooms': {'action': 'remove', 'max_reasonable': 10, 'reason': 'Plus de 10 chambres peu probable'},
    'bathrooms': {'action': 'remove', 'max_reasonable': 8, 'reason': 'Plus de 8 SdB peu probable'},
    'stories': {'action': 'remove', 'max_reasonable': 5, 'reason': 'Plus de 5 étages peu probable pour maison'},
    'parking': {'action': 'remove', 'max_reasonable': 10, 'reason': 'Plus de 10 places peu probable'}
}

for var, rule in treatment_rules.items():
    if var in df_clean.columns:
        if rule['action'] == 'remove':
            # Supprimer les valeurs déraisonnables
            initial_count = len(df_clean)
            df_clean = df_clean[df_clean[var] <= rule['max_reasonable']]
            removed = initial_count - len(df_clean)
            outliers_removed += removed
            
            print(f"🗑️ {var}: {removed} lignes supprimées (>{rule['max_reasonable']}) - {rule['reason']}")
            
        elif rule['action'] == 'cap':
            # Limiter les valeurs extrêmes aux percentiles
            lower_bound = df_clean[var].quantile(0.01)  # 1er percentile
            upper_bound = df_clean[var].quantile(0.99)  # 99e percentile
            
            initial_outliers = len(df_clean[(df_clean[var] < lower_bound) | (df_clean[var] > upper_bound)])
            
            # Appliquer le capping
            df_clean[var] = df_clean[var].clip(lower=lower_bound, upper=upper_bound)
            
            print(f"🧢 {var}: {initial_outliers} valeurs limitées aux percentiles [1%, 99%] - {rule['reason']}")

print(f"\n📊 RÉSUMÉ DU NETTOYAGE:")
print(f"   Lignes avant nettoyage: {len(df):,}")
print(f"   Lignes après nettoyage: {len(df_clean):,}")
print(f"   Lignes supprimées: {len(df) - len(df_clean):,} ({(len(df) - len(df_clean))/len(df)*100:.2f}%)")

# Vérifier l'impact sur les statistiques
print(f"\n💰 IMPACT SUR LES PRIX:")
print(f"   Prix moyen avant: {df['price'].mean():,.0f}")
print(f"   Prix moyen après: {df_clean['price'].mean():,.0f}")
print(f"   Écart-type avant: {df['price'].std():,.0f}")
print(f"   Écart-type après: {df_clean['price'].std():,.0f}")

# Utiliser les données nettoyées
df = df_clean.copy()
print(f"\n✅ Nettoyage des outliers terminé ! Dataset final: {df.shape}")

⚖️ DÉCISION DE TRAITEMENT DES OUTLIERS
🧢 price: 12 valeurs limitées aux percentiles [1%, 99%] - Prix extrêmes possibles dans l'immobilier
🧢 area: 12 valeurs limitées aux percentiles [1%, 99%] - Grandes propriétés possibles
🗑️ bedrooms: 0 lignes supprimées (>10) - Plus de 10 chambres peu probable
🗑️ bathrooms: 0 lignes supprimées (>8) - Plus de 8 SdB peu probable
🗑️ stories: 0 lignes supprimées (>5) - Plus de 5 étages peu probable pour maison
🗑️ parking: 0 lignes supprimées (>10) - Plus de 10 places peu probable

📊 RÉSUMÉ DU NETTOYAGE:
   Lignes avant nettoyage: 545
   Lignes après nettoyage: 545
   Lignes supprimées: 0 (0.00%)

💰 IMPACT SUR LES PRIX:
   Prix moyen avant: 4,766,729
   Prix moyen après: 4,751,146
   Écart-type avant: 1,870,440
   Écart-type après: 1,808,191

✅ Nettoyage des outliers terminé ! Dataset final: (545, 13)


## 🔧 Ingénierie des Fonctionnalités (Feature Engineering)

Créons de nouvelles variables pertinentes pour améliorer les performances du modèle.

In [6]:
# 1. Création de nouvelles variables
print("🔧 INGÉNIERIE DES FONCTIONNALITÉS")
print("=" * 40)

# Variables dérivées de base
df['price_per_sqft'] = df['price'] / df['area']
df['rooms_total'] = df['bedrooms'] + df['bathrooms']
df['area_per_room'] = df['area'] / df['rooms_total']
df['bathroom_bedroom_ratio'] = df['bathrooms'] / df['bedrooms']

print("✅ Variables dérivées de base créées:")
print("   • price_per_sqft: Prix par pied carré")
print("   • rooms_total: Nombre total de pièces")
print("   • area_per_room: Surface par pièce")
print("   • bathroom_bedroom_ratio: Ratio SdB/Chambres")

# 2. Variables d'équipements combinées
luxury_features = ['guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
df['luxury_score'] = df[luxury_features].sum(axis=1)
df['has_luxury'] = (df['luxury_score'] > 0).astype(int)

print("\n✅ Variables d'équipements créées:")
print("   • luxury_score: Score d'équipements de luxe (0-5)")
print("   • has_luxury: Présence d'au moins un équipement de luxe")

# 3. Catégorisation des tailles
def categorize_size(area):
    if area <= 5000:
        return 'small'
    elif area <= 8000:
        return 'medium'
    elif area <= 12000:
        return 'large'
    else:
        return 'very_large'

def categorize_price_range(price):
    if price <= 4000000:
        return 'budget'
    elif price <= 6000000:
        return 'mid_range'
    elif price <= 10000000:
        return 'premium'
    else:
        return 'luxury'

df['size_category'] = df['area'].apply(categorize_size)
df['price_category'] = df['price'].apply(categorize_price_range)

print("\n✅ Catégories créées:")
print("   • size_category: Catégorie de taille (small/medium/large/very_large)")
print("   • price_category: Gamme de prix (budget/mid_range/premium/luxury)")

# 4. Variables d'interaction importantes
df['area_bedrooms_interaction'] = df['area'] * df['bedrooms']
df['luxury_area_interaction'] = df['luxury_score'] * df['area']

print("\n✅ Variables d'interaction créées:")
print("   • area_bedrooms_interaction: Surface × Chambres")
print("   • luxury_area_interaction: Score luxe × Surface")

# 5. Aperçu des nouvelles variables
new_features = ['price_per_sqft', 'rooms_total', 'area_per_room', 'bathroom_bedroom_ratio',
                'luxury_score', 'has_luxury', 'size_category', 'price_category',
                'area_bedrooms_interaction', 'luxury_area_interaction']

print(f"\n📊 APERÇU DES NOUVELLES VARIABLES:")
for feature in new_features[:6]:  # Montrer les numériques d'abord
    if df[feature].dtype in ['int64', 'float64']:
        print(f"   {feature:25s}: {df[feature].mean():8.1f} (±{df[feature].std():6.1f})")

print(f"\n📊 DISTRIBUTION DES CATÉGORIES:")
for feature in ['size_category', 'price_category']:
    print(f"   {feature}:")
    counts = df[feature].value_counts()
    for cat, count in counts.items():
        pct = count/len(df)*100
        print(f"     {cat:12s}: {count:3d} ({pct:4.1f}%)")

print(f"\n✅ Feature engineering terminé ! Total variables: {df.shape[1]}")

🔧 INGÉNIERIE DES FONCTIONNALITÉS
✅ Variables dérivées de base créées:
   • price_per_sqft: Prix par pied carré
   • rooms_total: Nombre total de pièces
   • area_per_room: Surface par pièce
   • bathroom_bedroom_ratio: Ratio SdB/Chambres

✅ Variables d'équipements créées:
   • luxury_score: Score d'équipements de luxe (0-5)
   • has_luxury: Présence d'au moins un équipement de luxe

✅ Catégories créées:
   • size_category: Catégorie de taille (small/medium/large/very_large)
   • price_category: Gamme de prix (budget/mid_range/premium/luxury)

✅ Variables d'interaction créées:
   • area_bedrooms_interaction: Surface × Chambres
   • luxury_area_interaction: Score luxe × Surface

📊 APERÇU DES NOUVELLES VARIABLES:
   price_per_sqft           :    991.0 (± 340.8)
   rooms_total              :      4.3 (±   1.0)
   area_per_room            :   1254.7 (± 556.3)
   bathroom_bedroom_ratio   :      0.4 (±   0.2)
   luxury_score             :      1.1 (±   1.1)
   has_luxury               :      

## 📊 Division des Données (Train/Validation/Test)

Divisons nos données proprement pour l'entraînement et l'évaluation.

In [7]:
# 1. Préparation des variables pour la modélisation
print("📊 PRÉPARATION POUR LA DIVISION DES DONNÉES")
print("=" * 50)

# Variables cibles
target = 'price'
y = df[target].copy()

# Sélection des variables prédictives
# Exclure la cible et les catégories dérivées de la cible
features_to_exclude = [target, 'price_category']
feature_columns = [col for col in df.columns if col not in features_to_exclude]

# Séparer les variables numériques et catégorielles
numeric_features = df[feature_columns].select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df[feature_columns].select_dtypes(include=['object']).columns.tolist()

print(f"🎯 Variable cible: {target}")
print(f"📊 Features numériques: {len(numeric_features)}")
print(f"🏷️ Features catégorielles: {len(categorical_features)}")
print(f"📋 Total features: {len(numeric_features) + len(categorical_features)}")

print(f"\n🔢 FEATURES NUMÉRIQUES:")
for i, feat in enumerate(numeric_features, 1):
    print(f"   {i:2d}. {feat}")

if categorical_features:
    print(f"\n🏷️ FEATURES CATÉGORIELLES:")
    for i, feat in enumerate(categorical_features, 1):
        print(f"   {i:2d}. {feat}")
        unique_values = df[feat].unique()
        print(f"       Valeurs: {list(unique_values)}")

# Préparer X (features) avec encodage des variables catégorielles
X = df[numeric_features].copy()

# One-hot encoding pour les variables catégorielles
if categorical_features:
    print(f"\n🔄 ENCODAGE ONE-HOT:")
    for feat in categorical_features:
        # One-hot encoding
        dummies = pd.get_dummies(df[feat], prefix=feat, drop_first=True)
        X = pd.concat([X, dummies], axis=1)
        print(f"   ✅ {feat}: {len(dummies.columns)} nouvelles variables")

print(f"\n📊 DATASET FINAL POUR LA MODÉLISATION:")
print(f"   Shape X: {X.shape}")
print(f"   Shape y: {y.shape}")
print(f"   Total features après encodage: {X.shape[1]}")

📊 PRÉPARATION POUR LA DIVISION DES DONNÉES
🎯 Variable cible: price
📊 Features numériques: 20
🏷️ Features catégorielles: 1
📋 Total features: 21

🔢 FEATURES NUMÉRIQUES:
    1. area
    2. bedrooms
    3. bathrooms
    4. stories
    5. mainroad
    6. guestroom
    7. basement
    8. hotwaterheating
    9. airconditioning
   10. parking
   11. prefarea
   12. furnishingstatus
   13. price_per_sqft
   14. rooms_total
   15. area_per_room
   16. bathroom_bedroom_ratio
   17. luxury_score
   18. has_luxury
   19. area_bedrooms_interaction
   20. luxury_area_interaction

🏷️ FEATURES CATÉGORIELLES:
    1. size_category
       Valeurs: ['medium', 'large', 'very_large', 'small']

🔄 ENCODAGE ONE-HOT:
   ✅ size_category: 3 nouvelles variables

📊 DATASET FINAL POUR LA MODÉLISATION:
   Shape X: (545, 23)
   Shape y: (545,)
   Total features après encodage: 23


In [8]:
# 2. Division stratifiée des données
print("✂️ DIVISION DES DONNÉES")
print("=" * 30)

# Créer des strates basées sur les prix pour assurer une répartition équilibrée
price_bins = pd.qcut(y, q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
print("📊 Stratification basée sur les quartiles de prix:")
print(price_bins.value_counts().sort_index())

# Division principale: 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=price_bins
)

# Recréer les strates pour les données temporaires
price_bins_temp = pd.qcut(y_temp, q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Division train/validation: 75% train, 25% validation du reste
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,  # 25% de 80% = 20% du total
    random_state=42,
    stratify=price_bins_temp
)

# Résumé des divisions
print(f"\n📊 RÉPARTITION DES DONNÉES:")
print(f"   🏋️ Train: {X_train.shape[0]:3d} échantillons ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   ✅ Validation: {X_val.shape[0]:3d} échantillons ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"   🧪 Test: {X_test.shape[0]:3d} échantillons ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"   📊 Total: {len(X):3d} échantillons")

# Vérification de la répartition des prix dans chaque ensemble
print(f"\n💰 RÉPARTITION DES PRIX PAR ENSEMBLE:")
sets_info = {
    'Train': y_train,
    'Validation': y_val,
    'Test': y_test
}

for set_name, y_set in sets_info.items():
    print(f"   {set_name:10s}: {y_set.mean():8,.0f} (±{y_set.std():8,.0f}) [{y_set.min():7,.0f} - {y_set.max():8,.0f}]")

print(f"\n✅ Division des données terminée !")
print(f"📊 Features: {X_train.shape[1]}")
print(f"🎯 Prêt pour la normalisation et la modélisation")

✂️ DIVISION DES DONNÉES
📊 Stratification basée sur les quartiles de prix:
price
Q1    137
Q2    138
Q3    134
Q4    136
Name: count, dtype: int64

📊 RÉPARTITION DES DONNÉES:
   🏋️ Train: 327 échantillons (60.0%)
   ✅ Validation: 109 échantillons (20.0%)
   🧪 Test: 109 échantillons (20.0%)
   📊 Total: 545 échantillons

💰 RÉPARTITION DES PRIX PAR ENSEMBLE:
   Train     : 4,739,763 (±1,754,574) [1,870,400 - 10,542,000]
   Validation: 4,815,660 (±1,973,180) [1,870,400 - 10,542,000]
   Test      : 4,720,782 (±1,810,334) [1,870,400 - 10,150,000]

✅ Division des données terminée !
📊 Features: 23
🎯 Prêt pour la normalisation et la modélisation


## 🔄 Normalisation des Données

Normalisons les données pour optimiser les performances des algorithmes de machine learning.

In [9]:
# 1. Comparaison des méthodes de normalisation
print("🔄 NORMALISATION DES DONNÉES")
print("=" * 35)

# Tester différentes méthodes de normalisation sur un échantillon
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

print("🧪 COMPARAISON DES MÉTHODES DE NORMALISATION:")
sample_feature = 'price_per_sqft'

if sample_feature in X_train.columns:
    original_data = X_train[sample_feature]
    print(f"\n📊 Variable exemple: {sample_feature}")
    print(f"   Original: {original_data.mean():.1f} (±{original_data.std():.1f})")
    
    for name, scaler in scalers.items():
        # Normaliser la variable exemple
        transformed = scaler.fit_transform(original_data.values.reshape(-1, 1)).flatten()
        print(f"   {name:13s}: {transformed.mean():.3f} (±{transformed.std():.3f})")

# 2. Sélection et application de la méthode de normalisation
print(f"\n⚙️ APPLICATION DE LA NORMALISATION:")

# Utiliser RobustScaler car il est moins sensible aux outliers
scaler = RobustScaler()

# Normaliser les données d'entraînement
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

# Normaliser les données de validation et test avec le même scaler
X_val_scaled = pd.DataFrame(
    scaler.transform(X_val),
    columns=X_val.columns,
    index=X_val.index
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print(f"✅ RobustScaler appliqué à tous les ensembles")
print(f"   Train normalisé: {X_train_scaled.shape}")
print(f"   Validation normalisé: {X_val_scaled.shape}")
print(f"   Test normalisé: {X_test_scaled.shape}")

# 3. Vérification de la normalisation
print(f"\n📊 VÉRIFICATION DE LA NORMALISATION:")
print(f"   Variables avec moyenne ~0 et std ~1: {(abs(X_train_scaled.mean()) < 0.1).sum()}/{X_train_scaled.shape[1]}")

# Montrer quelques statistiques
print(f"\n📈 STATISTIQUES APRÈS NORMALISATION (Train):")
stats_sample = X_train_scaled.iloc[:, :5]  # Premières 5 variables
for col in stats_sample.columns:
    mean_val = stats_sample[col].mean()
    std_val = stats_sample[col].std()
    print(f"   {col:25s}: {mean_val:6.3f} (±{std_val:5.3f})")

print(f"\n✅ Normalisation terminée !")
print(f"📊 Données prêtes pour la modélisation")

🔄 NORMALISATION DES DONNÉES
🧪 COMPARAISON DES MÉTHODES DE NORMALISATION:

📊 Variable exemple: price_per_sqft
   Original: 1000.6 (±346.7)
   StandardScaler: 0.000 (±1.000)
   MinMaxScaler : 0.294 (±0.149)
   RobustScaler : 0.112 (±0.822)

⚙️ APPLICATION DE LA NORMALISATION:
✅ RobustScaler appliqué à tous les ensembles
   Train normalisé: (327, 23)
   Validation normalisé: (109, 23)
   Test normalisé: (109, 23)

📊 VÉRIFICATION DE LA NORMALISATION:
   Variables avec moyenne ~0 et std ~1: 5/23

📈 STATISTIQUES APRÈS NORMALISATION (Train):
   area                     :  0.215 (±0.751)
   bedrooms                 : -0.046 (±0.735)
   bathrooms                :  0.284 (±0.515)
   stories                  : -0.220 (±0.847)
   mainroad                 : -0.159 (±0.366)

✅ Normalisation terminée !
📊 Données prêtes pour la modélisation
✅ RobustScaler appliqué à tous les ensembles
   Train normalisé: (327, 23)
   Validation normalisé: (109, 23)
   Test normalisé: (109, 23)

📊 VÉRIFICATION DE LA NO

## 💾 Sauvegarde des Données Préparées

Sauvegardons toutes les données préparées pour la modélisation.

In [10]:
# 1. Sauvegarde des datasets
print("💾 SAUVEGARDE DES DONNÉES PRÉPARÉES")
print("=" * 40)

# Créer le dossier de sauvegarde
save_path = Path('../data/processed')
save_path.mkdir(parents=True, exist_ok=True)

# Sauvegarder les données non normalisées
X_train.to_csv(save_path / 'X_train_raw.csv', index=False)
X_val.to_csv(save_path / 'X_val_raw.csv', index=False)
X_test.to_csv(save_path / 'X_test_raw.csv', index=False)

y_train.to_csv(save_path / 'y_train.csv', index=False)
y_val.to_csv(save_path / 'y_val.csv', index=False)
y_test.to_csv(save_path / 'y_test.csv', index=False)

print("✅ Données brutes sauvegardées:")
print(f"   X_train_raw.csv: {X_train.shape}")
print(f"   X_val_raw.csv: {X_val.shape}")
print(f"   X_test_raw.csv: {X_test.shape}")
print(f"   y_train.csv, y_val.csv, y_test.csv")

# Sauvegarder les données normalisées
X_train_scaled.to_csv(save_path / 'X_train_scaled.csv', index=False)
X_val_scaled.to_csv(save_path / 'X_val_scaled.csv', index=False)
X_test_scaled.to_csv(save_path / 'X_test_scaled.csv', index=False)

print("\n✅ Données normalisées sauvegardées:")
print(f"   X_train_scaled.csv: {X_train_scaled.shape}")
print(f"   X_val_scaled.csv: {X_val_scaled.shape}")
print(f"   X_test_scaled.csv: {X_test_scaled.shape}")

# Sauvegarder le dataset complet avec les nouvelles features
df.to_csv(save_path / 'housing_final_cleaned.csv', index=False)
print(f"\n✅ Dataset complet sauvegardé:")
print(f"   housing_final_cleaned.csv: {df.shape}")

# Sauvegarder les métadonnées importantes
metadata = {
    'dataset_info': {
        'total_samples': len(df),
        'total_features': X_train.shape[1],
        'train_samples': X_train.shape[0],
        'val_samples': X_val.shape[0],
        'test_samples': X_test.shape[0]
    },
    'features': {
        'numeric_features': numeric_features,
        'categorical_features': categorical_features,
        'all_features': list(X_train.columns)
    },
    'target': target,
    'scaler_used': 'RobustScaler'
}

import json
with open(save_path / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f"\n✅ Métadonnées sauvegardées:")
print(f"   metadata.json")

# 2. Résumé final
print(f"\n🎯 RÉSUMÉ FINAL - ÉTAPE 2 TERMINÉE")
print("=" * 45)
print(f"✅ Données nettoyées: {len(df)} propriétés")
print(f"✅ Features créées: {X_train.shape[1]} variables")
print(f"✅ Outliers traités intelligemment")
print(f"✅ Données divisées (train/val/test)")
print(f"✅ Données normalisées avec RobustScaler")
print(f"✅ Tout sauvegardé dans {save_path}")

print(f"\n🚀 PROCHAINE ÉTAPE:")
print(f"   Étape 3 - Modélisation et évaluation")
print(f"   📊 {X_train.shape[0]} échantillons d'entraînement prêts")
print(f"   🎯 {X_train.shape[1]} features pour prédire les prix immobiliers")

💾 SAUVEGARDE DES DONNÉES PRÉPARÉES
✅ Données brutes sauvegardées:
   X_train_raw.csv: (327, 23)
   X_val_raw.csv: (109, 23)
   X_test_raw.csv: (109, 23)
   y_train.csv, y_val.csv, y_test.csv

✅ Données normalisées sauvegardées:
   X_train_scaled.csv: (327, 23)
   X_val_scaled.csv: (109, 23)
   X_test_scaled.csv: (109, 23)

✅ Dataset complet sauvegardé:
   housing_final_cleaned.csv: (545, 23)

✅ Métadonnées sauvegardées:
   metadata.json

🎯 RÉSUMÉ FINAL - ÉTAPE 2 TERMINÉE
✅ Données nettoyées: 545 propriétés
✅ Features créées: 23 variables
✅ Outliers traités intelligemment
✅ Données divisées (train/val/test)
✅ Données normalisées avec RobustScaler
✅ Tout sauvegardé dans ..\data\processed

🚀 PROCHAINE ÉTAPE:
   Étape 3 - Modélisation et évaluation
   📊 327 échantillons d'entraînement prêts
   🎯 23 features pour prédire les prix immobiliers
